[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/15b-numerical-operations-tensor.ipynb)

- ติดตั้ง lib pythainlp เพื่อใช้ในการตัดคำ และ อัพเดท lib gensim
- ทำการ mount google drive หรือ get dataset จาก git

In [2]:
!git clone https://github.com/superoutput/speech-pattern-training.git
!pip install pythainlp
!pip install gensim -U
!pip install matplotlib
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'speech-pattern-training'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 36 (delta 8), reused 28 (delta 3), pack-reused 0
Unpacking objects: 100% (36/36), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


[Document Referance](https://medium.com/@newnoi/%E0%B8%AA%E0%B8%AD%E0%B8%99%E0%B8%84%E0%B8%AD%E0%B8%A1%E0%B8%9E%E0%B8%B9%E0%B8%94%E0%B9%81%E0%B8%9A%E0%B8%9A%E0%B9%84%E0%B8%97%E0%B8%A2%E0%B9%86-%E0%B8%94%E0%B9%89%E0%B8%A7%E0%B8%A2-machine-learning-model-part2-2a1609af1bd7) สำหรับการทดสอบ

1. เตรียม Word Embedding model ภาษาไทย โดยใช้ gensim library ในการเทรน Word2Vec และนำ Corpus  มาจากบทความเกี่ยวกับโรคไข้หัดแมว(ข้อมูลส่วนนี้น้อยมากๆ ซึ่งทำให้ประสิทธิ์ภาพไม่ดี)

In [ ]:
import multiprocessing

from gensim.models import Word2Vec
from pythainlp.tokenize import word_tokenize
from gensim.models.word2vec import LineSentence

def preprocess(input_path, output_path):
  inputfile = open(input_path,'r')
  output = open(output_path, 'w')
  i = 0
  for line in inputfile.readlines():
    text = word_tokenize(line, keep_whitespace=False)
    output.write(' '.join(text) + "\n")
    i += 1
    if (i % 100 == 0) or (i <= 10 ):
        print("Saved " + str(i) + " articles")
  output.close()
  print("Finished Saved " + str(i) + " articles")


if __name__ == '__main__':
  inp = "/content/drive/MyDrive/Colab Notebooks/corpus.txt"
  outp = "/content/word.th.text"
  preprocess(inp, outp)
  model = Word2Vec(LineSentence(outp), vector_size=10, window=10, min_count=1,workers=multiprocessing.cpu_count())
  model.save('word_th.model')  
  print("Finished Saved Word Model")
  # wv_model = Word2Vec.load("/content/word_th.model")

หรืออีกวิธีสามารถ get word2vec จาก model thai2fit_wv [เลือกทดสอบด้วย model นี้]

In [ ]:
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from pythainlp.corpus import get_corpus_path

_MODEL_NAME = "thai2fit_wv"


def get_model() -> Word2VecKeyedVectors:
    """
    Get word vector model.

    :return: `gensim` word2vec model
    :rtype: gensim.models.keyedvectors.Word2VecKeyedVectors
    """
    path = get_corpus_path(_MODEL_NAME)
    return KeyedVectors.load_word2vec_format(path, binary=True)



wv_model = get_model()

2. การ train model 
- เตรียมข้อมูลเพื่อนำไป Train โดยข้อมูลที่ใช้เทรนจะเป็นประโยคที่เราจะสอนให้ตอบ
- สร้าง Encoder-Decoder LSTM model
- Train model โดยนำข้อมูลเข้าไปเรียนรู้

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras import Input,Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed
from pythainlp.tokenize import word_tokenize
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import matplotlib.pyplot as plt

function load_data โดย return question_list คือคำถาม, answer_list คือคำตอบ

In [7]:
def load_data(datafile): 
    data= pd.read_csv(datafile)
    question_list = [i.replace('\n', '') for i in data['2']]
    answer_list = [i.replace('\n', '') for i in data['3']]
    return question_list,answer_list

function preparingword จะทำการตัดคำในแต่ละประโยค

In [1]:
def preparingword(listword):
    word =[]
    for w in listword:
        word.append(word_tokenize(w,keep_whitespace=False))
    return word

function max_len นับจำนวน word ในประโยคที่เยอะที่สุดฃอง dataset

In [ ]:
def max_len(q, a):
    new_data = [len(i) for i in q]
    new_data.extend([len(i) for i in a])
    return max(new_data)

function padding_sequence จะทำการเพิ่มให้ทุกประโยค มีจำนวน word เท่ากัน โดยการเติม `<EOS>`ให้ครบตาม maxlen

In [9]:
def padding_sequence(listsentence, maxseq):
    dataset = []
    for s in listsentence:
        n = maxseq - len(s)
        if n>0:
            eos = ["<EOS>"]*n
            s.extend(eos)
            dataset.append(s)
        elif n<0:
            dataset.append(s[0:maxseq])
        else:
            dataset.append(s)
    return dataset

function word_index จะทำการหาค่าของ word vector กรณีไม่พบ word และ similar word ในwv_model จะทำการ default ให้เป็น `<eos>`

In [10]:
def word_index(listword):
    dataset = []
    for sentence in listword:
        tmp = []
        for w in sentence:
            tmp.append(word2idx(w))
        dataset.append(tmp)
    return np.array(dataset)

def word2idx(word):
    index = 0
    try:
        index = wv_model.key_to_index[word] 
    except:
        try:
            sim = similar_word(word)
            index = wv_model.key_to_index[sim] 
        except:
            index = wv_model.key_to_index["<eos>"] 
    return index

def similar_word(word):
    sim_word = wv_model.most_similar(word)
    try:
        return sim_word[0]
    except:
        return "<eos>"

function embedding_model จะทำการ define model โดยสร้าง Encoder-Decoder LSTM model 

In [13]:
def embedding_model():
    # define word embedding
    vocab_list = [(k, wv_model[k]) for k,v in wv_model.key_to_index.items()]
    embeddings_matrix = np.zeros((len(wv_model.key_to_index.items()) + 1, wv_model.vector_size))
    for i in range(len(vocab_list)):
        word = vocab_list[i][0]
        embeddings_matrix[i + 1] = vocab_list[i][1]

    embedding_layer = Embedding(input_dim=len(embeddings_matrix),
                                output_dim=300,
                                weights=[embeddings_matrix],
                                trainable=False,
                                name="Embedding"
                                )
    return embedding_layer,len(embeddings_matrix)

In [3]:
def ende_embedding_model(n_input, n_output, n_units):
    encoder_inputs = Input(shape=(None,), name="Encoder_input")

    encoder = LSTM(n_units,return_state=True, name='Encoder_lstm')
    Shared_Embedding,vocab_size = embedding_model()
    word_embedding_context = Shared_Embedding(encoder_inputs)
    encoder_outputs, state_h, state_c = encoder(word_embedding_context)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None,), name="Decoder_input")
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, name="Decoder_lstm")
    word_embedding_answer = Shared_Embedding(decoder_inputs)
    decoder_outputs, _, _ = decoder_lstm(word_embedding_answer, initial_state=encoder_states)
    decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax', name="Dense_layer"))
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    encoder_model = Model(encoder_inputs, encoder_states)
    decoder_state_input_h = Input(shape=(n_units,), name="H_state_input")
    decoder_state_input_c = Input(shape=(n_units,), name="C_state_input")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(word_embedding_answer, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    return model, encoder_model, decoder_model

In [ ]:
def train():
    # data = "/content/drive/MyDrive/Colab Notebooks/train_qa.txt"
    data = "/content/drive/MyDrive/dataset/itqa2.csv"
    max_word=10
    dataX = []
    dataY = []
    X1 = []
    X2 = []
    Y = []

    # load_data x คือคำถาม, y คือคำตอบ
    dataX,dataY = load_data(data)

    # preparingword และตัดคำ
    dataX = preparingword(dataX)
    dataY = preparingword(dataY)
    
    max_word = max_len(dataX, dataY)

    for sentence in dataX:
        X1.append(sentence)
    for sentence in dataY:
        Y.append(sentence)
    for sentence in dataY:
        add_start_word = ['_']
        add_start_word.extend(sentence)
        X2.append(add_start_word)

    # padding_sequence โดยเพิ่มให้ทุก sentence มีจำนวน word เท่ากัน
    X1 = padding_sequence(X1, maxlen)
    X2 = padding_sequence(X2, maxlen)
    Y = padding_sequence(Y, maxlen)

    # word_index หาค่าของ word vector
    X1 = word_index(X1)
    X2 = word_index(X2)
    Y = word_index(Y)

    # ทำ One-hot Encode ตามจำนวนของ word
    Y = to_categorical(Y, num_classes=51359)
    
    # define model
    train, infenc, infdec = ende_embedding_model(maxlen, maxlen, 256)
    train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    # train model
    train.fit([X1, X2], Y, epochs=300, batch_size=32)

    infenc.save_weights("/content/drive/MyDrive/model/model_enc_weight.h5")
    infenc.save("/content/drive/MyDrive/model/model_enc.h5")
    print("Saved model to disk")
    infdec.save_weights("/content/drive/MyDrive/model/model_dec_weight.h5")
    infdec.save("/content/drive/MyDrive/model/model_dec.h5")
    print("Saved model to disk")
    return train

model_his = train()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(model_his.history.history['acc'], label='acc')
plt.plot(model_his.history.history['loss'], label='loss')
plt.legend()
plt.show()

3. นำ Model ที่ได้มาใช้ โดยการป้อนคำถามเข้าไป

In [ ]:
def test_model():
    # # define model
    train, infenc, infdec = ende_embedding_model(maxlen, maxlen, 256)

    # load weights
    infenc.load_weights("/content/drive/MyDrive/model/model_enc.h5")
    infdec.load_weights("/content/drive/MyDrive/model/model_dec.h5")

    # start prediction
    while True:
        input_data = input()
        if input_data == 'end':
          break
        input_data = preparingword([input_data])
        input_data = padding_sequence(input_data, maxlen)
        input_data = word_index(input_data)
        target = predict_sequence(infenc, infdec, input_data, maxlen, 10)
        int_target = onehot_to_int(target)
        ans = invert(int_target)
        print(ans)
    return 0

def onehot_to_int(inputvector):
    return np.argmax(inputvector, axis=1)

def invert(inputlist):
    sentence = []
    for w in inputlist:
        sentence.append(wv_model.index_to_key[w])
    return ("".join(sentence).replace("<eos>"," "))

# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array(word_index("_"))
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = np.array([[np.argmax(yhat[0,0,:])]])
    return np.array(output)

test_model()

note... ใช้รันทดสอบ code

In [ ]:
# # for test code
# from gensim import models
# wv_model = models.Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/word_th.model")
# print(wv_model.wv.key_to_index.items())
# print(wv_model.wv.index_to_key[2])
aa = wv_model.most_similar('อีเมล')
print(aa)